In [ ]:
import pandas as pd
import os
import time
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa
import random
from util import sample_n_visitors, read_full_week
import seaborn as sns
import numpy as np
import math

In [ ]:
dataset_all = pq.ParquetDataset("data/hitdata7days")

In [5]:
columns = ['visitid','pageurl','pagename', 'pageeventvar2','pagetype', 'visitdatetime', 'hit_time_gmt', 'productlist', 'searchterms', 'searchresults', 'newvisit', 'post_evar27', 'evar28', 'evar83', 'ordernumber']
data_all = dataset_all.read(columns=columns).combine_chunks()

In [4]:
data_all.shape

(42730149, 16)

# Feature Engineering

## Adding memory features

### construct dictionary that maps product code 

### Track prices of products in the cart

### Track prices of products viewed

In [ ]:
cart_price_log = pd.read_pickle('cart_price_log.pkl')
pg_price_log = pd.read_pickle('pg_price_log.pkl')

## Assembling the rest of the features that will be in the training set

### Simpify pagenames

In [6]:
# simplyfied pagenames (before ':')
pgname_df = data_all['pagename'].to_pandas()

In [7]:
# check values before : in pagenames
pg_cat = pgname_df.str.split(':').str[0]

In [8]:
pg_cat_unique = pg_cat.unique()

In [9]:
len(pg_cat_unique)

49

There are 49 unique pagename categories, now let's see the distribution of these categories

In [10]:
pg_cat.value_counts()/len(pg_cat)

search                            2.417860e-01
pdp                               1.898416e-01
index                             1.778272e-01
checkout                          8.274738e-02
cart                              6.239199e-02
Cart_Interstitial                 6.166171e-02
mobile index                      5.798164e-02
Clothing                          1.356047e-02
Shoes                             1.155940e-02
Jewelry & Watches                 1.075604e-02
Electronics                       1.070949e-02
mobile shopping                   1.029189e-02
Toys                              9.280028e-03
Home                              7.206902e-03
Bed & Bath                        6.672057e-03
Video Games                       5.711354e-03
Furniture                         5.005459e-03
Featured Shops                    4.955869e-03
Kitchen                           4.605296e-03
Personal Care                     4.595561e-03
credit application                4.215735e-03
Gifts, Crafts

In [11]:
cat_ranking = pg_cat.value_counts()

In [12]:
sum(cat_ranking[:20])/len(pg_cat)

0.979147299486365

In [13]:
sum(cat_ranking[:10])/len(pg_cat)

0.9101133955793134

Top 10 pagename categories contains 91% of all pageviews, we'll use this as an arbitrary cutoff in order to simplify our dataset. We'll only keep the top ten categories and leave the rest as "other"

In [14]:
pg_cat_top10 = cat_ranking.index[:10]

In [15]:
pg_cat_simp = pg_cat.apply(lambda x: x if x in pg_cat_top10 else 'other')

In [16]:
# sanity check 
pg_cat_simp.value_counts()

search               10331552
pdp                   8111960
index                 7598581
other                 3840868
checkout              3535808
cart                  2666019
Cart_Interstitial     2634814
mobile index          2477564
Clothing               579441
Shoes                  493935
Jewelry & Watches      459607
Name: pagename, dtype: int64

### Channel

In [17]:
channel_df = data_all['post_evar27'].to_pandas()

In [18]:
channel_ct = channel_df.value_counts()

In [19]:
channel_df.isna().mean()

0.3640624796323551

Too many missing values, we'll not use this variable

### New visit

In [20]:
nv_df = data_all['newvisit'].to_pandas()

In [21]:
nv_df.isna().mean()

0.0

In [22]:
nv_counts = nv_df.value_counts()

In [23]:
nv_counts

0    41112195
1     1617954
Name: newvisit, dtype: int64

### Assemble the dataset

In [24]:
# visitid, pg_cat_simp, hittime, newvisit, cart_price_log, pg_price_log, ordernumber
data_2 = data_all.select(['visitid', 'hit_time_gmt', 'newvisit', 'ordernumber'])

In [25]:
data_2_df = data_2.to_pandas()

In [26]:
assert data_2_df.shape[0] == len(pg_cat_simp) == len(cart_price_log) == len(pg_price_log)

In [27]:
data_2_df['pg_cat'] = pg_cat_simp
data_2_df['cart_price_log'] = cart_price_log
data_2_df['pg_price_log'] = pg_price_log

In [29]:
data_2_df['hit_time_gmt'] = data_2_df['hit_time_gmt'].astype('int')

In [30]:
data_2_df['newvisit'] = data_2_df['newvisit'].astype('int')

In [32]:
# sort the dataset for ease of use
data_2_df_sorted = data_2_df.sort_values(['visitid', 'hit_time_gmt'], ascending=True)

### Add more features

#### Create labels for each row based on order number

In [34]:
has_order = ~data_2_df_sorted.groupby('visitid')['ordernumber'].apply(lambda x: all(x.isna()))

In [35]:
data_2_df_full_label = data_2_df_sorted.merge(has_order, on = 'visitid', suffixes = ('_original', '_full'))

In [37]:
# timedelta
timedelta = data_2_df_full_label.groupby('visitid')['hit_time_gmt'].diff()

In [39]:
# replace all timedelta = na with 0
td_na_idx = timedelta.isna()

In [40]:
timedelta[td_na_idx] = 0

In [41]:
data_2_df_full_label['timedelta'] = timedelta

In [42]:
# session time
session_time = data_2_df_full_label.groupby('visitid')['timedelta'].cumsum()

In [43]:
data_2_df_full_label['session_time'] = session_time

In [44]:
# pageview number

In [45]:
pg_count = data_2_df_full_label.groupby('visitid').cumcount()

In [46]:
data_2_df_full_label['pg_count'] = pg_count

#### cart price summary stats

In [47]:
# sum
sum_cart = data_2_df_full_label.groupby('visitid')['cart_price_log'].cumsum()

In [48]:
data_2_df_full_label['sum_cart'] = sum_cart

In [49]:
# avg

In [50]:
if_add_prod = pd.Series([0]*len(cart_price_log))

In [51]:
if_add_prod[data_2_df_full_label['cart_price_log'] > 0] = 1

In [52]:
if_add_prod[data_2_df_full_label['cart_price_log'] < 0] = -1

In [53]:
data_2_df_full_label['if_add_prod'] = if_add_prod

In [54]:
prod_in_cart = data_2_df_full_label.groupby('visitid')['if_add_prod'].cumsum()

In [55]:
data_2_df_full_label['prod_in_cart'] = prod_in_cart

In [56]:
is_prod_in_cart = prod_in_cart != 0

In [57]:
avg_cart = pd.Series([0]*data_2_df_full_label.shape[0])

In [58]:
d1 = data_2_df_full_label[is_prod_in_cart]

In [60]:
avg_cart_none0 = d1['sum_cart'] / d1['prod_in_cart']

In [75]:
avg_cart[avg_cart_none0.index] = avg_cart_none0

In [76]:
data_2_df_full_label['avg_cart'] = avg_cart

Maybe avg_cart shoulnd't be used since it gives a positive value even when products are only removed from cart

In [84]:
# min_cart
min_cart = data_2_df_full_label.groupby('visitid')['cart_price_log'].cummin()
data_2_df_full_label['min_cart'] = min_cart

In [86]:
# max_cart
max_cart = data_2_df_full_label.groupby('visitid')['cart_price_log'].cummax()
data_2_df_full_label['max_cart'] = max_cart

#### Viewed price summary stats

In [92]:
# sum_prod
sum_prod = data_2_df_full_label.groupby('visitid')['pg_price_log'].cumsum()

In [93]:
data_2_df_full_label['sum_prod'] = sum_prod

In [97]:
# avg_prod
if_view_prod = pd.Series([0]*len(pg_price_log))

In [98]:
if_view_prod[data_2_df_full_label['pg_price_log'] > 0] = 1

In [99]:
data_2_df_full_label['if_view_prod'] = if_view_prod

In [100]:
prod_viewed = data_2_df_full_label.groupby('visitid')['if_view_prod'].cumsum()
data_2_df_full_label['prod_viewed'] = prod_viewed

In [104]:
is_prod_viewed = prod_viewed != 0

In [108]:
avg_prod = pd.Series([0]*data_2_df_full_label.shape[0])

In [105]:
d2 = data_2_df_full_label[is_prod_viewed]

In [106]:
avg_prod_none0 = d2['sum_prod'] / d2['prod_viewed']

In [109]:
avg_prod[avg_prod_none0.index] = avg_prod_none0

In [110]:
data_2_df_full_label['avg_prod'] = avg_prod

In [116]:
# min_prod
min_prod = data_2_df_full_label.groupby('visitid')['pg_price_log'].cummin()
data_2_df_full_label['min_prod'] = min_prod

In [117]:
# max_prod
max_prod = data_2_df_full_label.groupby('visitid')['pg_price_log'].cummax()
data_2_df_full_label['max_prod'] = max_prod

## Now we have the dataset that's ready for training

In [122]:
features = ['visitid', 'newvisit', 'pg_cat', 'timedelta', 'session_time', 'pg_count', 'sum_cart', 'prod_in_cart', 'avg_cart', 'min_cart', 'max_cart', 'sum_prod', 'prod_viewed', 'avg_prod', 'min_prod', 'max_prod', 'ordernumber_full']
dataset_final = data_2_df_full_label[features]

In [ ]:
dataset_final